In [50]:
#import libraries

import statsmodels.api as sm
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.stats.anova import anova_lm
import numpy as np

In [51]:
#load crime data from data folder

df = pd.read_csv('../data/raw/crime.csv')

df.head()

,state,murder.rate,poverty,high.school,college,single.parent,unemployed,metropolitan,region
0,Alabama,7.4,14.7,77.5,20.4,26.0,4.6,70.2,South
1,Alaska,4.3,8.4,90.4,28.1,23.2,6.6,41.6,West
2,Arizona,7.0,13.5,85.1,24.6,23.5,3.9,87.9,West
3,Arkansas,6.3,15.8,81.7,18.4,24.7,4.4,49.0,South
4,California,6.1,14.0,81.2,27.5,21.8,4.9,96.7,West


In [52]:
unique_regions = df['region'].unique()
unique_regions

array(['South', 'West', 'Northeast', 'North Central'], dtype=object)

In [ ]:
# preprocessing 10/21 from Math 4400

# Create dummy variables for 'region'
region_dummies = pd.get_dummies(df['region'], drop_first=True)

# Join back to your main dataframe
df_encoded = pd.concat([df, region_dummies], axis=1)

In [54]:
# North Central is the baseline

df_encoded.head()

,state,murder.rate,poverty,high.school,college,single.parent,unemployed,metropolitan,region,Northeast,South,West
0,Alabama,7.4,14.7,77.5,20.4,26.0,4.6,70.2,South,False,True,False
1,Alaska,4.3,8.4,90.4,28.1,23.2,6.6,41.6,West,False,False,True
2,Arizona,7.0,13.5,85.1,24.6,23.5,3.9,87.9,West,False,False,True
3,Arkansas,6.3,15.8,81.7,18.4,24.7,4.4,49.0,South,False,True,False
4,California,6.1,14.0,81.2,27.5,21.8,4.9,96.7,West,False,False,True


In [55]:
# declare variables (drop murder rate from X, stage, and region (since we have region as a dummy variable) now)

X = df_encoded.drop(columns=['murder.rate', 'state', 'region'])
y = df_encoded['murder.rate']

In [60]:
# split the data in preparation of running a Linear Regression model
np.random.seed(214)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=214)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(40, 9) (10, 9) (40,) (10,)


In [57]:
# add a random seed
np.random.seed(425)

# Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [62]:
# Compute r squared on the training set
r_squared_train = model.score(X_train, y_train)
print(f"R-squared on training set: {r_squared_train:.4f}")

# Compute the r squared on the test set
r_squared = model.score(X_test, y_test)
print(f"R-squared on test set: {r_squared:.4f}")

R-squared on training set: 0.7162
R-squared on test set: 0.2726


In [ ]:
for i in df_encoded['Northeast', 'South', 'Weest']:
    if i == False:
        'region'[i] = 0
    elif i == True:
        'region'[i] = 1

poverty          float64
high.school      float64
college          float64
single.parent    float64
unemployed       float64
metropolitan     float64
Northeast           bool
South               bool
West                bool
dtype: object


In [58]:
# run anova

model_anova = sm.OLS(y_train, X_train).fit()
print(model_anova.summary())

# Compute the r^2 on the test set
r_squared = model_anova.score(X_test, y_test)
print(f"R-squared on test set: {r_squared:.4f}")

# Compute r^2 on the training set
r_squared_train = model_anova.score(X_train, y_train)
print(f"R-squared on training set: {r_squared_train:.4f}")

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [ ]:
# get rid of the not statistically significant variables

X_train_reduced = X.drop(columns=['poverty','college','unemployed'])
model_reduced = sm.OLS(y_train, X_).fit()
print(model_reduced.summary())

                                 OLS Regression Results                                
Dep. Variable:            murder.rate   R-squared (uncentered):                   0.919
Model:                            OLS   Adj. R-squared (uncentered):              0.904
Method:                 Least Squares   F-statistic:                              63.96
Date:                Tue, 21 Oct 2025   Prob (F-statistic):                    4.39e-17
Time:                        14:10:28   Log-Likelihood:                         -75.158
No. Observations:                  40   AIC:                                      162.3
Df Residuals:                      34   BIC:                                      172.4
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------